# Estimación por mínimos cuadrados
_Nicolás Villegas Vargas, María Camila Vásquez Correa_

Modelación Experimental, 2019-1

### ARMA: Datos artificiales

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.plotly as py
import plotly.graph_objs as go
import warnings
warnings.simplefilter('ignore')
%matplotlib inline
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

Generamos datos con los siguientes parámetros

In [3]:
A = np.matrix([[.75,-0.25],[1, 0]])
D = np.matrix([[.65,.35],[0,0]])

In [47]:
# Simulacion de datos
p = 2
q = 2
nobs = 250
X = np.zeros((250,p))
# condicion inicial
X[0] = np.array([-0.5,-1.4])
# Ruido
e = np.random.normal(size=nobs)

In [48]:
for i in range(1, nobs):
    X[i] = X[i-1]*A.transpose() + np.array([e[i-1],e[i]])*D.transpose()

In [6]:
trace1 = go.Scatter(
    y = X.T[0],
    x = np.arange(nobs)
)

trace2 = go.Scatter(
    y = X.T[1],
    x = np.arange(nobs)
)

data = [trace2, trace1]
py.iplot(data)

In [49]:
X = np.matrix(X)

### Ordinary Least Squares

### Estimación de parámetros

In [8]:
def ols(X, t):
    V = X[0].transpose()*X[1]
    G = X[0].transpose()*X[0]
    for i in range(1,t):
        V += X[i].transpose()*X[i+1]
        G += X[i].transpose()*X[i]
    return V.transpose()*np.linalg.inv(G)    

In [9]:
c1 = []
c2 = []
for t in range(5, nobs-1):
    c = ols(X, t)
    c1.append(c.A1[0])
    c2.append(c.A1[1])

In [10]:
a1 = [A.A1[0] for _ in range(len(c1))]
a2 = [A.A1[1] for _ in range(len(c2))]

In [11]:
trace = go.Scatter(
    y = a1,
    x = np.arange(nobs),
    name = 'Parametro a1')

trace1 = go.Scatter(
    y = c1,
    x = np.arange(nobs),
    name = 'Estimacion a1')

trace2 = go.Scatter(
    y = a2,
    x = np.arange(nobs),
    name = 'Parametro a2')

trace3 = go.Scatter(
    y = c2,
    x = np.arange(nobs),
    name = 'Estimacion a2')

layout = go.Layout(
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickcolor='rgb(204, 204, 204)',
        tickwidth=2,
        ticklen=5,
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    yaxis=dict(
        showgrid=False,
        zeroline=False,
        showline=False,
        showticklabels=False,
    ),
    autosize=False,
    margin=dict(
        autoexpand=False,
        l=100,
        r=20,
        t=110,
    )
)

annotations = []
annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05,
                              xanchor='left', yanchor='bottom',
                              text='Estimación con mínimos cuadrados ordinarios',
                              font=dict(family='Arial',
                                        size=30,
                                        color='rgb(37,37,37)'),
                              showarrow=False))

layout['annotations'] = annotations

layout['annotations'] = annotations
traces = [trace, trace1, trace2, trace3]

fig = go.Figure(data=traces, layout=layout)
py.iplot(fig)


## Instrumental Variables

In [12]:
def IVols(X,t):
    V = X[0].transpose()*X[2]
    G = X[2].transpose()*X[1]
    for i in range(1,t):
        V += X[i].transpose()*X[i+2]
        G += X[i+2].transpose()*X[i]
    return V.transpose()*np.linalg.inv(G)    

In [13]:
X = np.matrix(X)
c1 = []
c2 = []
for t in range(5, nobs-1):
    c = IVols(X, t)
    c1.append(c.A1[0])
    c2.append(c.A1[1])

In [14]:
trace = go.Scatter(
    y = a1,
    x = np.arange(nobs),
    name = 'Parametro a1')

trace1 = go.Scatter(
    y = c1,
    x = np.arange(nobs),
    name = 'Estimacion a1')

trace2 = go.Scatter(
    y = a2,
    x = np.arange(nobs),
    name = 'Parametro a2')

trace3 = go.Scatter(
    y = c2,
    x = np.arange(nobs),
    name = 'Estimacion a2')

layout = go.Layout(
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickcolor='rgb(204, 204, 204)',
        tickwidth=2,
        ticklen=5,
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    yaxis=dict(
        showgrid=False,
        zeroline=False,
        showline=False,
        showticklabels=False,
    ),
    autosize=False,
    margin=dict(
        autoexpand=False,
        l=100,
        r=20,
        t=110,
    )
)

annotations = []
annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05,
                              xanchor='left', yanchor='bottom',
                              text='Estimación con variables instrumentales',
                              font=dict(family='Arial',
                                        size=30,
                                        color='rgb(37,37,37)'),
                              showarrow=False))

layout['annotations'] = annotations

layout['annotations'] = annotations
traces = [trace, trace1, trace2, trace3]

fig = go.Figure(data=traces, layout=layout)
py.iplot(fplt.plot(y)ig)


### Recursive Least Squares

In [19]:
def RLS(X, t):
    V = X[0].transpose()*X[1]
    G = X[0].transpose()*X[0]
    R = np.linalg.inv(G)
    C = V0.T*R0
    for i in range(1, t):
        R -= (R*X[i].T*X[i]*R)/(1+X[i]*R*X[i].T)
        C += (X[i+1].T - C*X[i].T)*X[i]*R
    return C

In [20]:
c1 = []
c2 = []
for t in range(50, nobs):
    c = RLS(X, t)
    c1.append(c.A1[0])
    c2.append(c.A1[1])

In [21]:
trace = go.Scatter(
    y = a1,
    x = np.arange(nobs),
    name = 'Parametro a1')

trace1 = go.Scatter(
    y = c1,
    x = np.arange(nobs),
    name = 'Estimacion a1')

trace2 = go.Scatter(
    y = a2,
    x = np.arange(nobs),
    name = 'Parametro a2')

trace3 = go.Scatter(
    y = c2,
    x = np.arange(nobs),
    name = 'Estimacion a2')

layout = go.Layout(
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickcolor='rgb(204, 204, 204)',
        tickwidth=2,
        ticklen=5,
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    yaxis=dict(
        showgrid=False,
        zeroline=False,
        showline=False,
        showticklabels=False,
    ),
    autosize=False,
    margin=dict(
        autoexpand=False,
        l=100,
        r=20,
        t=110,
    )
)

annotations = []
annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05,
                              xanchor='left', yanchor='bottom',
                              text='Estimación con mínimos cuadrados recursivos',
                              font=dict(family='Arial',
                                        size=30,
                                        color='rgb(37,37,37)'),
                              showarrow=False))

layout['annotations'] = annotations

layout['annotations'] = annotations
traces = [trace, trace1, trace2, trace3]

fig = go.Figure(data=traces, layout=layout)
py.iplot(fig)


In [23]:
from __future__ import print_function
import numpy as np
from scipy import stats
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.sandbox.regression.predstd import wls_prediction_std
from statsmodels.iolib.table import (SimpleTable, default_txt_fmt)
import pandas as pd

In [61]:
y = list(X.T[1])[:-1]
x1 = X.T[0][:-1]
x2 = X.T[0][1:]

In [24]:
nsample = 50
x = np.linspace(0, 20, nsample)
X = np.column_stack((x, (x - 5)**2))
X = sm.add_constant(X)
beta = [5., 0.5, -0.01]
sig = 0.5
w = np.ones(nsample)
w[nsample * 6//10:] = 3
y_true = np.dot(X, beta)
e = np.random.normal(size=nsample)
y = y_true + sig * w * e 
X = X[:,[0,1]]

In [51]:
X1 = np.column_stack((x1,x2))

In [72]:
list(X.T)

[matrix([[-5.00000000e-01,  1.91660688e-01,  9.26244898e-01,
           4.04091761e-01, -6.88882021e-01,  4.02503994e-02,
           1.68458433e+00,  1.57300164e+00,  1.59795410e+00,
           8.46764818e-01,  9.80364027e-02,  5.42836776e-01,
           3.61487030e-01,  7.25629727e-04, -1.61975413e-02,
           8.15505487e-01, -1.23541580e-01, -6.67349397e-01,
          -4.22510187e-01,  9.33912282e-01,  7.50267626e-01,
           4.80725126e-02,  4.87647746e-01,  6.78450159e-01,
          -6.50311244e-01, -8.50402320e-01, -3.60136789e-01,
           7.52850679e-01,  1.57080614e+00, -1.86059784e-01,
          -7.64733031e-01, -9.33968860e-03,  4.18026915e-01,
          -5.94174431e-01, -1.82874022e+00, -7.85063156e-01,
           2.66800346e-01,  1.14062566e+00,  2.37278613e-01,
           4.87010569e-01,  5.22541164e-01,  8.12678461e-01,
           7.84936296e-01, -7.24258287e-01, -1.59543162e+00,
          -1.20551374e+00, -1.33891482e+00, -1.05933100e+00,
           2.29645517e-0